In [165]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math
import numpy as np

# train test split 
from sklearn.model_selection import train_test_split

# Kfold
from sklearn.model_selection import KFold

# regex
import re
 

In [103]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

In [104]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [105]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [106]:
iris_X.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3]}, inplace = True)


In [107]:
iris_y.rename(columns = {0:'target'}, inplace = True) 

In [108]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [109]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [9]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [10]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [11]:
def split_information(data, gain_attribute):
    res = entropy(data, gain_attribute)
    if(res==0):
        res=0.00000001
    return res

In [12]:
def gain_ratio(data, gain_attribute, target_attribute):
    res = information_gain(data, gain_attribute, target_attribute) / split_information(data, gain_attribute)
    return res

In [13]:
def gain_ratio_continous(data, gain_attribute, target_attribute):
    res = information_gain_continous(data, gain_attribute, target_attribute)[0] / split_information(data, gain_attribute)
    return res,information_gain_continous(data, gain_attribute, target_attribute)[1]

In [14]:
def best_attribute(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if information_gain(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = information_gain(data, i, target_attribute)
                    gain_attribute['name'] = i
            else:
                if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                    gain_attribute['name'] = i
                

    return gain_attribute['name']

In [15]:
import math

class Node:
    def __init__(self, attribute=None, label=None, vertex=None):
        self.attribute = attribute
        self.label = label
        self.vertex = vertex
        self.children = {}
        self.most_common_label = None
    
    def set_most_common_label(self, most_common_label):
        self.most_common_label = most_common_label
        
    def get_most_common_label(self):
        return self.most_common_label
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
        
    def setVertex(self, vertex):
        self.vertex = vertex
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def getLabel(self):
        return self.label
    
    def getVertex(self):
        return self.vertex

In [16]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [17]:
def most_common_label_target(data,target_attribute):
    most_comm = None
    occ = 0;
    for i in data[target_attribute].unique():
        if data[data[target_attribute] == i].shape[0] >  occ:
            most_comm = i
            occ = data[data[target_attribute] == i].shape[0]
    return most_comm

In [18]:
def id3(data, target_attribute, is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute,is_IG)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
            node.set_most_common_label(most_common_label_target(data,target_attribute))        
            
    return node

In [19]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +str(node.label))
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +str(i))
            print_tree(node.children[i],depth+1)        

In [20]:
def copy_tree(node):
    temp_node = Node()
    if node.label is not None: 
        temp_node.setLabel(node.label)
    else:
        temp_node.setAttribute(node.attribute)
        for i in node.children:
            temp= Node()
            temp= node.children[i]
            temp_node.addChildren(i, temp)
    return temp_node

In [21]:
def check_tree(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        for i in node.children:
            if i==data.loc[index,node.attribute]:
                check_tree(node.children[i],data,index,result, target_attribute)

In [22]:
def pred(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree(model,data[i:i+1],i,result, target_attribute)
    data = {target_attribute:result}
    return pd.DataFrame(data)
    

In [23]:
# hasn't handle after universal entropy
def information_gain_continous(data, gain_attribute, target_attribute):
    gain_result = 0
    save_boundary = -1
    min_entropy = 999999 
    data = data.sort_values(gain_attribute)
    data=data.reset_index().drop('index',axis=1)
    length_data = len(data)
    for i in range(length_data):
        if (i!=length_data-1):
            if (data.loc[i,target_attribute] != data.loc[i+1,target_attribute]):
                temp_boundary = (data.loc[i,gain_attribute] + data.loc[i+1,gain_attribute])/2
                parsed_data_upper = data.loc[data[gain_attribute]>=temp_boundary]
                parsed_data_lower = data.loc[data[gain_attribute]<temp_boundary]
                len_parsed = len(parsed_data_upper)               
                temp_entropy = float(len_parsed)/length_data * entropy(parsed_data_upper, target_attribute) + float((length_data-len_parsed))/length_data * entropy(parsed_data_lower, target_attribute)     
                if temp_entropy < min_entropy:
                    #print(gain_attribute, temp_boundary, temp_entropy)
                    min_entropy = temp_entropy 
                    save_boundary = temp_boundary

    gain_result += entropy(data,target_attribute) + (-1 * min_entropy)
    return gain_result,save_boundary

In [24]:
information_gain_continous(iris, 'petal length (cm)', 'target')

(0.9182958340544894, 2.45)

In [25]:
def best_attribute_c45(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': '',
        'boundary': -99999999
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = information_gain_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if information_gain(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = information_gain(data, i, target_attribute)
                            gain_attribute['name'] = i
            else:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = gain_ratio_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                            gain_attribute['name'] = i

    return gain_attribute['name'],round(gain_attribute['boundary'],2)

In [26]:
best_attribute_c45(iris,'target',False)

('petal width (cm)', 0.8)

In [27]:
def c45(data, target_attribute,is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
        else:
            node.addChildren('>='+str(bound),c45(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG))
            node.addChildren('<'+str(bound),c45(data.loc[data[best_attribute_]<bound],target_attribute,is_IG))
    return node

In [28]:
def c45_prun(data, target_attribute,is_IG,vertex_=None):
    node = Node(vertex = vertex_)
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45_prun(data.loc[data[best_attribute_]==i],target_attribute,is_IG,i))
        else:
            node.addChildren('>='+str(bound),c45_prun(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG,'>='+str(bound)))
            node.addChildren('<'+str(bound),c45_prun(data.loc[data[best_attribute_]<bound],target_attribute,is_IG,'<'+str(bound)))
    return node

In [29]:
c45(iris, "target",True)

In [30]:
print_tree(c45(iris, "target",True),0)

[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.75
        [sepal width (cm)]
------------>=3.15
            [sepal length (cm)]
---------------->=6.05
                    2
----------------<6.05
                    1
------------<3.15
                2
--------<1.75
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [31]:
def check_tree_c45(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        else:
            for i in node.children:
                #print(i)
                if i[0]=='<':
                    #print(i,2)
                    bound=float(i[1:])
                    if bound>data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                elif i[0]=='>':
                    bound=float(i[2:])
                    #print(i,1)
                    if bound<=data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                else:
                    if i==data.loc[index,node.attribute]:
                        check_tree(node.children[i],data,index,result, target_attribute)
                    

In [32]:
def pred_c45(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree_c45(model,data[i:i+1],i,result, target_attribute)
    data = { target_attribute: data[target_attribute]
            ,'prediction':result}
    return pd.DataFrame(data)
    

In [33]:
p=iris

In [34]:
p=p.drop('target',axis=1)

In [35]:
p=p.reset_index().drop('index',axis=1)

In [36]:
p['sepal width (cm)'] = None

In [37]:
def accuracy(pred,data):
    cnt = 0
    for i in range(len(pred)):
        if pred.loc[i] == data.loc[i]:
            cnt+=1
    return cnt*100/len(pred)

In [38]:
def get_data_validate(data):
    data_column = data.columns
    data_10 = pd.DataFrame(columns=data_column)
    data_90 = pd.DataFrame(columns=data_column)
    #print(data_column)
    count_10 = 0
    count_90 = 0
    
    for i in range(data.shape[0]):
        if(i%10 == 1):
            # Pass the row elements as key value pairs to append() function 
            data_10 = data_10.append(data.loc[[i]] , ignore_index=True)
        else:
            data_90 = data_90.append(data.loc[[i]] , ignore_index=True)
    return data_10, data_90

In [39]:
def prune_tree(node, attribute, vertex):
    if(node.label is not None):
        return node,0
    elif(node.attribute == attribute and node.vertex == vertex):
        node.children = {}
        node.attribute = None
        node.label = node.most_common_label
        
        return node,1
    else:
        for i in node.children:
            a,b = prune_tree(node.children[i], attribute, vertex)
            
        return node,b

In [40]:
def post_pruning(data, node, current_node, children_node, target_attribute):
    if children_node == 0:
        return node
    else:
        for i in current_node.children:
            if current_node.vertex is None: 
                #print(i,1)
                post_pruning(data, node, current_node.children[i], children_node-1, target_attribute)
            else:
                if current_node.attribute is not None:
                    data10, data90 = get_data_validate(data)
                    #print(current_node.attribute,2)
                    save = copy_tree(current_node)
                    temp_node,c = prune_tree(node, current_node.attribute, current_node.vertex)
                    train=data90.drop(target_attribute,axis=1)
                    #print_tree(node,0)
                    temp_node_pred = pred_c45(train, node, target_attribute)
                    #print(temp_node_pred)
                    temp_node_accuracy = accuracy(temp_node_pred[target_attribute], data90[target_attribute])
                    #print(temp_node_accuracy)
                    if temp_node_accuracy == 100:
                        post_pruning(data, temp_node, current_node, children_node, target_attribute)
                    else :
                        
                        node.addChildren(current_node.vertex, save)
                        #print_tree(node,0)


                        post_pruning(data, node, save.children[i], children_node, target_attribute)
        return node

In [41]:





############################################          ANN              ###################################################





def sigmoid(x):
    return 1/(1+math.exp(-x))

In [42]:
def sign(x):
    if (x>0.5):
        return 1
    else:
        return 0

In [43]:
def count_error(target, output):
    return 0.5*(target-output)*(target-output)

In [44]:
class Neuron:
    def __init__(self, out=None, w=None, is_used=None, error=None):
        self.out = out
        self.w = []
        self.is_used = is_used
        self.error = 0
        self.deltaW = []
        
    def set_out(self, out):
        self.out = out
        
    def get_out(self):
        return self.out
    
    def add_deltaW(self, value):
        self.deltaW.append(value)
        
    def get_deltaW(self, index):
        return self.deltaW[index]
    
    def get_arrdW(self):
        return self.deltaW
    
    def set_deltaW(self, index, value):
        self.deltaW[index] = value
        
    def add_w(self, value):
        self.w.append(value)
        
    def set_w(self, index, value):
        self.w[index] = value
    
    def get_w(self, index):
        return self.w[index]
    
    def get_arrW(self):
        return self.w
    
    def set_is_used(self, is_used):
        self.is_used = is_used
        
    def get_is_used(self):
        return self.is_used
    
    def set_error(self, error):
        self.error = error
    
    def get_error(self):
        return self.error

In [45]:
x = [Neuron() for i in range (5)]
y = [x for i in range(5)]

In [46]:
def LenCol(arr):
    return len(arr)
def LenRow(inp,hid,out):
    return max(inp,max(hid)+1,out)
def MakeMatrix(row,col):
    return [[Neuron() for i in range (col)] for j in range(row)]

In [47]:
def printMatrixMLP(matrix):
    for i in range(len(matrix)):
        temp_matrix_out = []
        for j in range(len(matrix[i])):
            temp_matrix_out.append(matrix[i][j].get_arrW())
        print(temp_matrix_out)

In [48]:
def convTarget(output,unique):
    res = []
    for i in range(unique):
        if (i!=output):
            res.append(0)
        else:
            res.append(1)
    return res
    

In [49]:
def resetDeltaWeight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].set_deltaW(k,0)

In [50]:
def initW(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].add_w(np.random.uniform(-1,1)) 
                mat[j][i].add_deltaW(0)

In [51]:
def feedforward(mat,layer):
    for i in range(1,len(layer)):
        if (i!=len(layer)-1):
            mat[layer[i]][i].set_out(1)
        for j in range(layer[i]):
            net = 0;
            for k in range(layer[i-1]):
                #print(k,i-1,j)
                net = net + mat[k][i-1].get_out()*mat[k][i-1].get_w(j)
#             if (i!=len(layer)-1):
            mat[j][i].set_out(sigmoid(net))
#             else:
#                 mat[j][i].set_out(sign(sigmoid(net)))

In [52]:
def get_error_total(matrix, n_output, target_arr):
    output_col = len(matrix[0])-1
    total_error = 0
    for i in range(n_output):
        total_error += 0.5*(target_arr[i] - matrix[i][output_col].get_out())**2
    
    return total_error

In [53]:
def set_output_error(matrix, row, col, target_value):
    output_value = matrix[row][col].get_out()
    #print(output_value, target_value)
#     print(output_value * (1 - output_value) * (target_value - output_value))
    matrix[row][col].set_error(output_value * (1 - output_value) * (target_value - output_value))

In [54]:
def set_hidden_error(matrix, row, col):
    output_value = matrix[row][col].get_out()
    delta_weight_error = 0
    for i in range(len(matrix[row][col].w)):
        delta_weight_error += matrix[row][col].get_w(i) * matrix[i][col+1].get_error()
    matrix[row][col].set_error(output_value * (1 - output_value) * delta_weight_error)

In [55]:
def update_delta_weight(matrix, row, col, idx , learning_rate):
    #print(matrix[i][col+1].get_error(),matrix[row][col].get_out(),row,col)
    #print(row,col+1)
    delta_weight = learning_rate * matrix[row][col].get_error() * matrix[idx][col-1].get_out()
    #print(delta_weight)
    
    current_weight = matrix[idx][col-1].get_deltaW(row)
    matrix[idx][col-1].set_deltaW(row, current_weight + delta_weight)
    #print(idx,col-1,row,current_weight + delta_weight)

In [56]:
def update_weight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                curr_weight = mat[j][i].get_w(k)
                delta_weight = mat[j][i].get_deltaW(k)
                mat[j][i].set_w(k,curr_weight + delta_weight) 

In [57]:
def backpropagation(matrix, layer, learning_rate, target_arr):
    for i in range(len(layer)-1, 0, -1):
        if i == len(layer)-1:
            for j in range(layer[i]):
                set_output_error(matrix, j, i, target_arr[j])
        else:
            for j in range(layer[i]):
                set_hidden_error(matrix, j, i)
        for j in range(layer[i-1]):
            for k in range(layer[i]):
                update_delta_weight(matrix, k, i, j, learning_rate)

In [58]:
hidden_node = [3,4]
def MLP(data,target, hidden_node, epochs, learning_rate):
    output_node = data[target].nunique()
    input_node = len(data.columns)
    hidden_node.insert(0,input_node)
    hidden_node.append(output_node)
    layer_node = hidden_node
    mlp = MakeMatrix(LenRow(input_node,hidden_node,output_node),LenCol(hidden_node))
    initW(mlp,layer_node)
    train = data.drop(target,axis=1)
    batch = len(data)/10;
    for k in range(epochs):
        error = 0
        for i in range(int(batch)):
            for j in range(10):
                for index,col in enumerate(train.columns):
                    mlp[index][0].set_out(data[col][j+i*10])
                mlp[input_node-1][0].set_out(1)
                feedforward(mlp,layer_node)
                backpropagation(mlp, layer_node, learning_rate, convTarget(data[target][j+i*10],output_node))
                error += get_error_total(mlp, len(convTarget(data[target][j+i*10],output_node)), convTarget(data[target][j+i*10],output_node))
            update_weight(mlp,layer_node)
            resetDeltaWeight(mlp,layer_node)
        print(error)
    print()
    print()
    printMatrixMLP(mlp)

    return mlp,layer_node

In [59]:
def predMyMLP(data, target, model, output_layer):
    out = []
    mlp = model[0]
    for i in range(0,data.shape[0],1):
        for index,col in enumerate(data.columns):
            mlp[index][0].set_out(data[col][i])
        mlp[len(data.columns)-1][0].set_out(1)
        feedforward(mlp,model[1])
        output_col = len(mlp[0])-1
        #print(output_col)
        mx=0
        res=-1
        for j in range(output_layer):
            # print(mlp[j][output_col].get_out())
            if mlp[j][output_col].get_out()>mx:
                mx = mlp[j][output_col].get_out()
                res = j
        # print()
        out.append(res)
    data = { target: data[target],
             'prediction':out}
    return pd.DataFrame(data)


# Testing

In [61]:
class_list = iris['target'].unique()

# Create confusion matrix from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
#    Array              class_list
# Output:
#    pandas.DataFrame   confusion_matrix
#
def confusion_matrix(prediction_data, actual_column, prediction_column, class_list):
    actual_data = pd.Categorical(prediction_data[actual_column], categories=class_list)
    prediction_data = pd.Categorical(prediction_data[prediction_column], categories=class_list)
    confusion_matrix = pd.crosstab(actual_data, prediction_data, rownames=['Actual'], colnames=['Predicted'], dropna=False)
    return confusion_matrix

In [62]:
# Get accuracy from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
# Output:
#    float              accuracy
#
def testing_accuracy(prediction_data, actual_column, prediction_column):
    count = 0
    for index, row in prediction_data.iterrows():
        if(row[actual_column] == row[prediction_column]):
            count = count + 1
    return round(count/prediction_data.shape[0],2)
            


## Train Test Split

In [112]:
iris_X.drop('index',axis=1,inplace=True);
iris_y.drop('index',axis=1,inplace=True);

In [182]:
################################# Testing DTL
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.1, random_state=42)

In [183]:
def mergeData(X_train, X_test, y_train, y_test):
    X_train = X_train.reset_index()
    y_train = y_train.reset_index()
    X_test = X_test.reset_index()
    y_test = y_test.reset_index()
    trainData = X_train.merge(y_train)
    testData = X_test.merge(y_test)
    trainData.drop("index",axis=1,inplace=True)
    testData.drop("index",axis=1,inplace=True)
    return trainData,testData

In [184]:
trainData,testData = mergeData(X_train, X_test, y_train, y_test)

In [210]:
model = c45(trainData, "target",True)
prediction_dtl = pred_c45(testData, model, "target")
print(prediction_dtl)

    target  prediction
0        0           0
1        0           0
2        0           0
3        1           1
4        1           1
5        1           1
6        1           1
7        1           1
8        1           1
9        2           2
10       2           1
11       2           2
12       2           2
13       2           2
14       2           2


In [211]:
print(confusion_matrix(prediction_dtl,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          3  0  0
1          0  6  0
2          0  1  5


In [212]:
testing_accuracy(prediction_dtl,'target','prediction')

0.93

In [205]:
hidden_node = [4,4,2]
x = MLP(trainData, 'target', hidden_node,1500,0.1)

54.24009267476858
50.69027305959694
48.774771008886766
47.424813392190885
46.33687064051906
45.46135896247162
44.81714574333522
44.39656460200716
44.14953483148432
44.01387620493781
43.94047542099432
43.8987339803696
43.87206199361655
43.852215427641006
43.83533907360268
43.819717568448766
43.804623881337655
43.7897658391097
43.7750321127412
43.76038048293433
43.745791062032005
43.731248055562446
43.716733489191014
43.70222560833309
43.687698872462505
43.67312432311861
43.65846989199607
43.643700530343445
43.62877815384566
43.61366142810185
43.59830542024421
43.58266113332469
43.56667492981272
43.55028784138662
43.53343475449115
43.51604345445119
43.49803350463685
43.47931493063479
43.45978667198494
43.439334755209806
43.41783013103243
43.395126105280994
43.3710552764623
43.34542587283631
43.318017357593305
43.28857514209526
43.25680421392685
43.222361448792824
43.18484633357562
43.14378978334114
43.09864069024318
43.04874980181551
42.99335049811548
42.93153603429601
42.86223285574472


14.313295500802008
14.316261470171918
14.318995329545936
14.321547453449119
14.323971693087849
14.326325113336832
14.328668227173877
14.331065762351852
14.333587993604114
14.336312705946174
14.339327904804488
14.342735442922164
14.346655765910123
14.351233916093504
14.356646578866261
14.363108776557626
14.370875469914397
14.380224732549074
14.391389780881022
14.404374768692106
14.418583172200377
14.432386976016497
14.443357278566307
14.449843018656914
14.452233017822484
14.451880254494908
14.449727686591297
14.446179586286728
14.441305467450174
14.43494871629177
14.426846374246406
14.416954674594765
14.406043431909378
14.395550588540909
14.385531875549253
14.375203517779225
14.365237135337425
14.354848677172583
14.345495222841986
14.334019314682362
14.328483020288964
14.305592583747169
14.337376997791594
14.174122860785115
14.360335885669611
14.060846684550576
14.333641186984714
14.00384349245072
14.334830383413516
13.954060659506892
14.350954399115423
13.90848808600033
14.359261153605

13.189574382327862
13.187883186900795
13.18619040030748
13.184495934763005
13.182799703273323
13.181101619657191
13.17940159856741
13.177699555511227
13.1759954068699
13.174289069917426
13.172580462838667
13.1708695047463
13.169156115697254
13.16744021670809
13.16572172976968
13.164000577860953
13.162276684961881
13.16054997606553
13.15882037718933
13.157087815385585
13.155352218750837
13.153613516434929
13.151871638648647
13.150126516670946
13.148378082855244
13.146626270634737
13.14487101452719
13.143112250138714
13.1413499141667
13.139583944402144
13.137814279731009
13.136040860134823
13.13426362669056
13.132482521569582
13.130697488035931
13.128908470443745
13.127115414233891
13.125318265929844
13.123516973132677
13.121711484515346
13.119901749816153
13.118087719831241
13.116269346406536
13.114446582428634
13.112619381814866
13.110787699502714
13.10895149143802
13.107110714562584
13.105265326800813
13.103415287045268
13.101560555141488
13.099701091871735
13.097836858937738
13.09596

11.966000330004965
11.964148050257215
11.96230473056912
11.96047026955905
11.958644567112364
11.956827524366842
11.955019043698059
11.953219028704813
11.951427384194512
11.949644016168662
11.94786883180829
11.94610173945955
11.94434264861914
11.942591469920073
11.94084811511738
11.93911249707382
11.937384529745803
11.935664128169387
11.933951208446276
11.932245687729985
11.930547484212221
11.928856517109205
11.92717270664814
11.92549597405392
11.923826241535854
11.922163432274552
11.920507470408939
11.918858281023406
11.917215790135113
11.915579924681321
11.913950612507097
11.912327782352905
11.91071136384246
11.909101287470707
11.907497484592009
11.905899887408308
11.904308428957622
11.902723043102545
11.901143664518992
11.899570228685006
11.898002671869728
11.896440931122555
11.894884944262444
11.893334649867203
11.891789987263198
11.890250896514935
11.888717318414953
11.887189194473793
11.885666466910067
11.88414907864077
11.882636973271632
11.881130095087666
11.879628389043763
11.8

In [206]:
prediction_ann = predMyMLP(testData, 'target', x, 3)
print(prediction_ann)

    target  prediction
0        0           0
1        0           0
2        0           0
3        1           1
4        1           1
5        1           1
6        1           1
7        1           1
8        1           1
9        2           2
10       2           2
11       2           2
12       2           2
13       2           2
14       2           2


In [207]:
print(confusion_matrix(prediction_ann,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          3  0  0
1          0  6  0
2          0  0  6


In [208]:
testing_accuracy(prediction_ann,'target','prediction')

1.0

## CrosVal

# DTL

In [161]:
kf = KFold(n_splits=10,shuffle=True)

In [195]:
accuracy_DTL_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    model = c45(trainData, "target",True)
    prediction_dtl = pred_c45(testData, model, "target")
    accuracy_DTL_KF.append(testing_accuracy(prediction_dtl,'target','prediction'))

In [196]:
accuracy_DTL_KF

[0.87, 0.93, 1.0, 1.0, 0.8, 0.93, 0.93, 0.87, 0.93, 1.0]

In [197]:
round(np.mean(accuracy_DTL_KF)*100,2)

92.6

# MLP

In [213]:
accuracy_MLP_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    hidden_node = [4,4,2]
    # itu yang 200nya diganti 1000 biar akurasinya tinggi
    model = MLP(trainData, 'target', hidden_node,1500,0.1)
    prediction_ann = predMyMLP(testData, 'target', x, 3)
    accuracy_MLP_KF.append(testing_accuracy(prediction_ann,'target','prediction'))

54.35294246051127
50.827267125522354
49.137715339188595
48.01593877886239
47.09092497256902
46.24472579729487
45.47633438223157
44.84027263143187
44.38206061581664
44.09619926840435
43.936855801000675
43.852423554013455
43.805564374961946
43.774775346257464
43.74905272824341
43.72282242766306
43.69268802536238
43.655522389916804
43.60749511041214
43.544794327582196
43.46721661298048
43.38000828330068
43.28730900863365
43.186620283498975
43.071318619512134
42.933765428400676
42.7656377984512
42.55694485865384
42.29466066171305
41.96099892490996
41.53096469663964
40.96837601391218
40.21934811337904
39.209781397165884
37.875857349792625
36.21110667196459
34.280328720945136
32.227380799474446
30.251598840766754
28.529883114554398
27.1450496582
26.08515121594845
25.289864737742164
24.692220281225925
24.237228307716155
23.884759385299695
23.6067488074617
23.38370846868847
23.201981719548158
23.0518457425721
22.9262534059578
22.820005170204674
22.72919986082114
22.650865160113575
22.582704738

13.055908439087933
13.050507981649313
13.045281621584802
13.040246413827886
13.035423971452119
13.030841855296675
13.026535400734131
13.022550304633478
13.018946462877716
13.015803834049349
13.01323160966311
13.011382862047133
13.010478345441218
13.010845280486905
13.012977912644464
13.017613548636792
13.02572242916073
13.03784631956682
13.051041502157503
13.056309755831002
13.057814195327808
13.056504600268111
13.057394325108303
13.047613056713024
13.074939139688567
12.964071678597511
13.152455193279842
13.085318017819265
13.100916711249102
13.079678402137967
13.24600316691887
13.22167408571379
13.174370690381881
13.101078818909887
12.935783511288417
13.537607938069726
13.085837142042125
13.446132228390827
13.110132089320702
13.372438015390877
13.187570342848854
13.023132590936067
13.562418005137431
13.198035664885325
13.024473280821955
13.40029520116139
13.131251733740473
13.44846998781709
13.051304533044432
13.275509172518177
13.158970616350292
13.15533832953002
13.16725781335302
13

12.563442156223518
12.560854268174609
12.55826808943857
12.55568363092664
12.553100901174622
12.550519906231084
12.547940649551355
12.545363131898966
12.542787351254729
12.540213302735083
12.537640978519747
12.535070367790233
12.532501456679134
12.529934228231468
12.527368662377928
12.524804735921075
12.522242422534097
12.519681692772766
12.517122514100413
12.514564850925673
12.512008664653044
12.509453913745565
12.506900553799262
12.50434853762872
12.501797815362876
12.499248334550437
12.496700040273526
12.494152875269057
12.491606780056202
12.489061693069043
12.48651755079323
12.483974287905344
12.481431837413709
12.478890130799396
12.47634909815638
12.47380866832977
12.471268769050646
12.468729327066818
12.466190268268695
12.463651517809067
12.461113000216265
12.458574639500304
12.456036359250941
12.453498082727888
12.450959732942517
12.448421232730787
12.445882504817874
12.443343471873861
12.44080405656104
12.438264181572993
12.435723769665564
12.433182743680373
12.43064102656085
1

11.016589782141999
11.013304557037793
11.009501090958237
11.00562530574024
11.002100612693809
10.999060088542539
10.996378168071391
10.993891364582293
10.991508003690798
10.989190274609795
10.986920500483663
10.984687635547637
10.982483695535425
10.980302524994825
10.978139186444638
10.97598960549004
10.97385034498063
10.971718452691473
10.969591354290689
10.967466775544608
10.965342684320971
10.963217246362916
10.961088790987828
10.958955784066331
10.95681680650206
10.95467053692153
10.952515737675183
10.950351243471877
10.948175952165993
10.945988817324075
10.943788842307294
10.941575075661342
10.939346607670876
10.937102567966635
10.934842124114411
10.932564481133559
10.930268881920034
10.927954608560137
10.925620984538806
10.923267377852744
10.920893205048444
10.918497936206448
10.916081100895115
10.913642295111378
10.91118118921861
10.90869753687575
10.90619118493226
10.903662084234051
10.901110301249943
10.898536030382317
10.895939606772124
10.893321519345047
10.890682423775354
1

19.852221571983087
19.81356585509759
19.774516362974694
19.735083614537285
19.69527594417495
19.65509873332605
19.61455365132661
19.573637997874595
19.53234426722847
19.49066005073183
19.448568357564607
19.406048371738688
19.363076591462868
19.319628232665345
19.275678736423284
19.23120520841417
19.18618763733939
19.14060978159268
19.094459667918446
19.047729700127352
19.000416419981104
18.95251999020325
18.90404348011631
18.854992030522265
18.805371961307284
18.755189868444404
18.70445174112666
18.653162117466064
18.601323289554628
18.548934565422716
18.49599159563454
18.442485774994587
18.388403734577523
18.3337269461396
18.278431470685646
18.222487896832053
18.16586153414063
18.10851295309639
18.050398997208443
17.991474432080786
17.93169443562763
17.871018160964596
17.809413597695293
17.746863885738826
17.683375056248
17.618984845766267
17.553771740523022
17.487862819808093
17.42143846514963
17.35473189716209
17.288022127141254
17.221620376032966
17.15585194084952
17.09103699398338

...

11.967311055919481
11.964294634932793
11.961309450443974
11.958354167054262
11.95542754009992
11.95252840726424
11.949655681169231
11.94680834280898
11.943985435709472
11.941186060718053
11.938409371340656
11.935654569557423
11.932920902057283
11.93020765684124
11.9275141601506
11.924839773682455
11.922183892060612
11.91954594053287
11.916925372870919
11.91432166945091
11.911734335496154
11.909162899465324
11.906606911571565
11.904065942420031
11.901539581751823
11.899027437284996
11.896529133642968
11.894044311362785
11.891572625975982
11.889113747155427
11.886667357922823
11.884233153911248
11.881810842678416
11.879400143066384
11.877000784603846
11.874612506947738
11.872235059360769
11.86986820022257
11.867511696571333
11.865165323673947
11.862828864622434
11.86050210995481
11.858184857298435
11.855876911034345
11.853578081981125
11.851288187096792
11.849007049197738
11.846734496693104
11.844470363334096
11.84221448797675
11.839966714357683
11.837726890881537
11.835494870419922
11.8

11.0941873481003
11.093028919590262
11.091872852720831
11.090719130910395
11.089567737824492
11.088418657370335
11.087271873691583
11.086127371163185
11.084985134386425
11.083845148184006
11.082707397595346
11.08157186787209
11.08043854447349
11.079307413062171
11.07817845949988
11.077051669843362
11.075927030340345
11.074804527425673
11.073684147717495
11.07256587801356
11.071449705287627
11.070335616685965
11.06922359952391
11.068113641282556
11.06700572960554
11.065899852295763
11.064795997312485
11.063694152768129
11.062594306925499
11.06149644819483
11.060400565131047
11.05930664643101
11.058214680930888
11.057124657603536
11.056036565556004
11.05495039402703
11.05386613238467
11.05278377012391
11.051703296864396
11.050624702348154
11.049547976437466
11.04847310911267
11.047400090470074
11.04632891071995
11.045259560184508
11.044192029295953
11.04312630859459
11.042062388726936
11.041000260443937
11.039939914599113
11.038881342146931
11.037824534140999
11.036769481732433
11.035716

17.28751122237691
17.271363793488007
17.25571845458073
17.240587578773635
17.22598382738622
17.21192015842507
17.1984098262434
17.18546636679954
17.173103562922464
17.16133538413232
17.15017589585901
17.139639133353658
17.1297389361875
17.120488739979496
17.111901322881543
17.103988505382272
17.09676080317289
17.090227034162716
17.084393882256386
17.07926542222709
17.074842611960317
17.07112276051224
17.068098982831096
17.065759654619256
17.064087883639722
17.06306101675159
17.062650205018468
17.062820052264943
17.063528375286257
17.064726106276495
17.066357369509436
17.068359764261164
17.070664883549536
17.073199092434724
17.075884579287358
17.078640677762117
17.081385436090756
17.084037384805434
17.086517426744336
17.08875074834954
17.090668633976712
17.09221006013896
17.093322957544956
17.09396505567472
17.094104264070626
17.09371859005375
17.092795636308857
17.091331756472147
17.089330967755807
17.086803725402827
17.08376565622946
17.080236331644905
17.076238139022824
17.0717952883

13.307932505794017
13.306493682984529
13.30511409887133
13.303792778498863
13.30252878833689
13.30132123501429
13.30016926391316
13.299072057625478
13.29802883427569
13.297038845713503
13.296101375582047
13.295215737267855
13.294381271739756
13.293597345284942
13.292863347151785
13.292178687109441
13.291542792935672
13.29095510784503
13.290415087870679
13.289922199213656
13.289475915574345
13.289075715481623
13.28872107963583
13.288411488281696
13.288146418628743
13.287925342335745
13.287747723077077
13.287613014207945
13.287520656545773
13.28747007628465
13.28746068305913
13.287491868172953
13.287563003007925
13.287673437626431
13.287822499580784
13.288009492940523
13.288233697548117
13.28849436851153
13.288790735940873
13.289122004934617
13.289487355819706
13.28988594464802
13.290316903950925
13.29077934375232
13.291272352840055
13.291795000295487
13.29234633728109
13.292925399086894
13.293531207438045
13.294162773068512
13.29481909856862
13.29549918151868
13.29620201792597
13.296926

13.0065531968269
13.004074655686285
13.001600900767544
12.999131909811956
12.996667660865713
12.99420813227188
12.991753302662465
12.989303150950942
12.986857656324778
12.984416798238446
12.981980556406532
12.979548910797119
12.977121841625314
12.974699329347128
12.972281354653449
12.969867898464177
12.967458941922715
12.965054466390452
12.962654453441571
12.9602588848579
12.957867742624094
12.955481008922686
12.953098666129733
12.950720696810109
12.948347083713388
12.945977809769458
12.943612858084673
12.941252211937742
12.938895854776092
12.93654377021204
12.934195942019299
12.931852354129552
12.92951299062899
12.927177835755142
12.924846873893681
12.922520089575428
12.920197467473297
12.917878992399446
12.915564649302567
12.913254423264993
12.910948299500285
12.908646263350505
12.906348300283831
12.904054395892096
12.90176453588848
12.899478706105228
12.897196892491428
12.894919081110881
12.892645258139979
12.890375409865754
12.888109522683811
12.885847583096476
12.883589577710877
1

In [214]:
accuracy_MLP_KF

[0.93, 0.93, 1.0, 0.93, 0.93, 1.0, 1.0, 1.0, 0.93, 1.0]

In [215]:
round(np.mean(accuracy_MLP_KF)*100,2)

96.5